### Intro

In [11]:
import csv
import os
from datetime import datetime

USERS_FILE = "source_files/users.csv"
MOVIES_FILE = "source_files/movies.csv"
BORROW_FILE = "source_files/borrowings.csv"


# ================================
# CSV HELPERS
# ================================

def load_csv(filename):
    if not os.path.exists(filename):
        return []

    with open(filename, newline="", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        return list(reader)


def save_csv(filename, data, fieldnames):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)


def generate_new_id(records):
    if not records:
        return 1
    return max(int(r["id"]) for r in records) + 1


### User Management

In [12]:
# ================================
# USER MANAGEMENT
# ================================

def add_user():
    users = load_csv(USERS_FILE)
    name = input("Enter user name: ").strip()
    if not name:
        print("Name cannot be empty.")
        return

    user_id = generate_new_id(users)
    users.append({"id": user_id, "name": name})
    save_csv(USERS_FILE, users, ["id", "name"])

    print(f"User added with ID: {user_id}")


def view_users():
    users = load_csv(USERS_FILE)
    if not users:
        print("No users found.")
        return

    print("\n--- USERS ---")
    for u in users:
        print(f"{u['id']}: {u['name']}")
    print("-------------")


### Movie Management

In [13]:
# ================================
# MOVIE MANAGEMENT
# ================================

def add_movie():
    movies = load_csv(MOVIES_FILE)

    title = input("Enter movie title: ").strip()
    director = input("Enter director: ").strip()
    year = input("Enter release year: ").strip()
    copies = input("Enter number of copies: ").strip()

    if not title or not director or not year or not copies.isdigit():
        print("Invalid input.")
        return

    movie_id = generate_new_id(movies)

    movies.append({
        "id": movie_id,
        "title": title,
        "director": director,
        "year": year,
        "copies": copies
    })

    save_csv(MOVIES_FILE, movies,
             ["id", "title", "director", "year", "copies"])

    print(f"Movie added with ID: {movie_id}")


def view_movies():
    movies = load_csv(MOVIES_FILE)
    if not movies:
        print("No movies found.")
        return

    print("\n--- MOVIES ---")
    for m in movies:
        print(f"{m['id']} | {m['title']} | {m['director']} | {m['year']} | Copies: {m['copies']}")
    print("--------------")

### Borrowing System - Borrow

In [15]:
# ================================
# BORROWING SYSTEM
# ================================

def borrow_movies():
    users = load_csv(USERS_FILE)
    movies = load_csv(MOVIES_FILE)
    borrowings = load_csv(BORROW_FILE)

try:
        user_id = input("User ID: ").strip()
except ValueError:
        print("Invalid User ID.")
        return

if not any(u["id"] == user_id for u in users):
    print("User not found.")
    return

    movie_ids = input("Enter Movie IDs (comma separated): ")
    movie_ids = [m.strip() for m in movie_ids.split(",")]

    movies_dict = {m["id"]: m for m in movies}
    unavailable = []

# Check availability
for mid in movie_ids:
    movie = movies_dict.get(mid)
    if not movie:
        unavailable.append(f"{mid} (not found)")
    elif int(movie["copies"]) <= 0:
        unavailable.append(f"{mid} ({movie['title']} - no copies)")

 if unavailable:
    print("\nUnavailable:")
    for u in unavailable:
        print(" -", u)
    return
        


# Borrow
for mid in movie_ids:
        movies_dict[mid]["copies"] = str(int(movies_dict[mid]["copies"]) - 1)

    # Find existing borrowing record
    record = next((b for b in borrowings if b["user_id"] == user_id), None)

 if record:
        current = record["movie_ids"].split(",")
        record["movie_ids"] = ",".join(current + movie_ids)
 else:
        borrowings.append({
            "user_id": user_id,
            "movie_ids": ",".join(movie_ids),
            "date": datetime.now().strftime("%Y-%m-%d")
        })

    save_csv(MOVIES_FILE, movies_dict.values(),
             ["id", "title", "director", "year", "copies"])
    save_csv(BORROW_FILE, borrowings, ["user_id", "movie_ids", "date"])

    print("Movies borrowed successfully!")


def return_movies():
    movies = load_csv(MOVIES_FILE)
    movies_dict = {m["id"]: m for m in movies}
    borrowings = load_csv(BORROW_FILE)

user_id = input("User ID: ").strip()

    record = next((b for b in borrowings if b["user_id"] == user_id), None)
 if not record:
        print("User has no borrowings.")
        return

    movie_ids = [m.strip() for m in input("Enter Movie IDs to return: ").split(",")]

    borrowed = record["movie_ids"].split(",")

for mid in movie_ids:
        if mid in borrowed:
            borrowed.remove(mid)
            movies_dict[mid]["copies"] = str(int(movies_dict[mid]["copies"]) + 1)

    if borrowed:
        record["movie_ids"] = ",".join(borrowed)
    else:
        borrowings = [b for b in borrowings if b["user_id"] != user_id]

    save_csv(MOVIES_FILE, movies_dict.values(),
             ["id", "title", "director", "year", "copies"])
    save_csv(BORROW_FILE, borrowings,
             ["user_id", "movie_ids", "date"])

    print("Movies returned successfully!")


def list_borrowed_movies():
    movies = load_csv(MOVIES_FILE)
    movies_dict = {m["id"]: m for m in movies}
    borrowings = load_csv(BORROW_FILE)

user_id = input("User ID: ").strip()

    record = next((b for b in borrowings if b["user_id"] == user_id), None)
if not record:
        print("No borrowings found.")
        return

    print("\n--- Borrowed Movies ---")
    print("Date:", record["date"])
    for mid in record["movie_ids"].split(","):
        print(f"{mid}: {movies_dict[mid]['title']}")
    print("-----------------------")



IndentationError: unindent does not match any outer indentation level (<string>, line 34)

### Menu

In [ ]:
# ================================
# MENU
# ================================
def main_menu():
    while True:
        print("\n===== MOVIE MANAGEMENT SYSTEM (CSV) =====")
        print("1. Add User")
        print("2. View Users")
        print("3. Add Movie")
        print("4. View Movies")
        print("5. Borrow Movies")
        print("6. Return Movies")
        print("7. List Borrowed Movies")
        print("8. Exit")

        choice = input("Choose option: ")

        if choice == "1":
            add_user()
        elif choice == "2":
            view_users()
        elif choice == "3":
            add_movie()
        elif choice == "4":
            view_movies()
        elif choice == "5":
            borrow_movies()
        elif choice == "6":
            return_movies()
        elif choice == "7":
            list_borrowed_movies()
        elif choice == "8":
            break
        else:
            print("Invalid option.")


main_menu()
